In [ ]:
# !pip install fuzzywuzzy
# !pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 17.1 MB/s eta 0:00:00


In [1]:
import warnings
import evaluate

warnings.filterwarnings(
    "ignore",
    message="Using the latest cached version of the module.*",
    category=UserWarning  # Or possibly a more specific warning category
)

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
ASU_key = os.environ.get("ASU_key") 
endpoint_url = os.environ.get("endpoint_url") 

import requests
start_ii = 0

import functions as fun

headers = {
    "Authorization": f"Bearer {ASU_key}",
    "Content-Type": "application/json"
}

#################################   
import pandas as pd

df = pd.read_csv(r'C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data\\silver_data_4_16_2025.csv',
                 dtype={'title': 'str', 
                        'url': 'str', 
                        'document_type': 'str',
                        'chunked_word_count':'int64',
                        'orig_word_count':'int64',
                        'contex':'str',
                        'question':'str',
                        'answer':'str',
                        'filename':'str',
                        'section':'str'})
df = df.reset_index(drop=True)
df.rename( columns={'contex': 'context'},inplace=True)

df['question'] = df['question'].str.replace('\n', ' ')
df['answer'] = df['answer'].str.replace('\n', ' ')
df['context'] = df['context'].str.replace('\n', ' ')

# Identify the rows where 'context' is numeric and remove them
numeric_context_mask = pd.to_numeric(df['context'], errors='coerce').notna()
# Filter out those rows
df = df[~numeric_context_mask].copy()

df['similarity_score'] = 0.0
df['NLTK_bleu'] = 0.0
df['sacrebleu_bleu'] = 0.0
df['Cosine'] = 0.0
df['rouge1'] = 0.0
df['rouge2'] = 0.0
df['rougeL'] = 0.0
df['rougeLsum'] = 0.0
df['sBURT'] = 0.0

df['ragas_bleu'] = 0.0
df['ragas_precision_reference'] = 0.0
df['ragas_faith'] = 0.0
df['ragas_fact'] = 0.0
df['regas_sem_sim'] = 0.0
df['regas_rogue'] = 0.0

df = df[['title', 'url', 'document_type', 'chunked_word_count', 'orig_word_count', 'context', 'question', 'answer', 'filename']]
df.shape, df.columns

Index(['title', 'url', 'document_type', 'chunked_word_count',
       'orig_word_count', 'context', 'question', 'answer', 'filename'],
      dtype='object')

In [23]:
df.head()

,title,url,document_type,chunked_word_count,orig_word_count,context,question,answer,filename
0,ASU Library Research Data Repository,https://dataverse.asu.edu/dataverse/asulibrary...,web page,500,814,Apr 19 2024 Physical and Computational Thermal...,"Besides parking infrastructure, what other res...",Post fire debris flow risk and roadway vulnera...,processed_data_thread_10_start_115100_500_rows...
1,ASU Library Research Data Repository,https://dataverse.asu.edu/dataverse/asulibrary...,web page,500,814,Apr 19 2024 Physical and Computational Thermal...,"Besides parking infrastructure, what other res...","Another research area is neuroscience, specifi...",processed_data_thread_10_start_115100_500_rows...
2,ASU Library Research Data Repository,https://dataverse.asu.edu/dataverse/asulibrary...,web page,500,814,Apr 19 2024 Physical and Computational Thermal...,"Besides parking infrastructure, what other res...",One dataset focuses on characterizing human ex...,processed_data_thread_10_start_115100_500_rows...
3,ASU Library Research Data Repository,https://dataverse.asu.edu/dataverse/asulibrary...,web page,500,814,Apr 19 2024 Physical and Computational Thermal...,What are some of the methodologies used to cre...,The post fire debris flow risk assessment uses...,processed_data_thread_10_start_115100_500_rows...
4,ASU Library Research Data Repository,https://dataverse.asu.edu/dataverse/asulibrary...,web page,500,814,Apr 19 2024 Physical and Computational Thermal...,What are some of the methodologies used to cre...,"In creating the parking inventory for Phoenix,...",processed_data_thread_10_start_115100_500_rows...


In [18]:
df.dtypes

title                         object
url                           object
document_type                 object
chunked_word_count             int64
orig_word_count                int64
context                       object
question                      object
answer                        object
filename                      object
similarity_score             float64
Cosine                       float64
sacrebleu_bleu                object
NLTK_bleu                    float64
chrf                          object
sBURT                        float32
rouge1                       float64
rouge2                       float64
rougeL                       float64
rougeLsum                    float64
ragas_bleu                   float64
ragas_precision_reference    float64
ragas_faith                  float64
ragas_fact                   float64
regas_sem_sim                float64
regas_rogue                  float64
dtype: object

In [24]:
# print(df[pd.to_numeric(df['answer'], errors='coerce').notna()])
print(df['context'][pd.to_numeric(df['context'], errors='coerce').notna()])

Series([], Name: context, dtype: object)


In [ ]:
import datetime
print(datetime.datetime.now())
df['similarity_score'] = df.apply(lambda row: fun.calculate_fuzzy_similarity(str(row['answer']), str(row['context'])), axis=1)
print(datetime.datetime.now())

2025-04-17 14:07:11.082065


In [5]:
#############################################################################

if start_ii > 0:
    df = pd.read_csv('C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data\\qa_results_RAGAS_evaluated.csv')

for ii in range(start_ii, 2): #len(df['question'])):
    print(ii)

    questiony = df['question'].iloc[ii]
    answery = df['answer'].iloc[ii]
    contexty = df['context'].iloc[ii]


    ###########################################################################
    #### old school
    ###########################################################################
    similarity_score = fun.calculate_fuzzy_similarity(answery, contexty)
    df.loc[ii, 'similarity_score'] = similarity_score
    # print(f"Fuzzy Similarity Score: {similarity_score}")

    tfidf_similarity = fun.calculate_tfidf_cosine_similarity(answery, contexty)
    df.loc[ii, 'Cosine'] = tfidf_similarity
    # print(f"TF-IDF Cosine Similarity: {tfidf_similarity:.2f}")

    ## sacrebleu bleu
    df.loc[ii, 'sacrebleu_bleu'] = fun.BLEU_SIM(A = answery, B =  contexty)

    ## NLTK bleu
    bleu_score = fun.calculate_bleu_score(answery, contexty)
    df.loc[ii, 'NLTK_bleu'] = bleu_score
    # print(f"BLEU Score: {bleu_score:.2f}")

    ## charcter level f score
    df.loc[ii, 'chrf'] = fun.FSCORE_SIM(A = answery, B =  contexty)

    ## sBURT
    df.loc[ii, 'sBURT'] = fun.sBERT(tokens1 = answery, tokens2 = contexty)

    ## rogue calcs
    results = fun.rouge_SIM(A =  contexty, B = answery)

    df.loc[ii, 'rouge1'] = results['rouge1']
    df.loc[ii, 'rouge2'] = results['rouge2']
    df.loc[ii, 'rougeL'] = results['rougeL']
    df.loc[ii, 'rougeLsum'] = results['rougeLsum']
    
    ###########################################################################
    #### RAGAS
    ###########################################################################

    ##################################
    ### Bluescore
    payload_blue = {
        "model_provider": "openai",
        "model_name": "gpt-4o",
        "metric": "bleuscore",
        "parameters": {
            "user_input": questiony,
            "response": answery,
            "reference": contexty,
        }
    }
    try:
        response = requests.post(endpoint_url, headers=headers, json=payload_blue)
        response.raise_for_status()
        result = response.json()
        print("result:", result)
        #### SAVE BLEU score
        df.loc[ii,'ragas_bleu'] = result['score']
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

    ##################################
    ## Context Precision With Reference
    payload_precision_reference = {
        "model_provider": "openai",
        "model_name": "gpt-4o",
        "metric": "context_precision_with_reference",
        "parameters": {
            "user_input": questiony,
            "reference": answery,
            "retrieved_contexts": [
                contexty,
            ]
        }
    }
    try:
        response = requests.post(endpoint_url, headers=headers, json=payload_precision_reference)
        response.raise_for_status()
        result = response.json()
        print("result:", result)
        #### SAVE Precision With Reference score
        df.loc[ii,'ragas_precision_reference'] = result['score']
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

    #################################
    ### RAGAS Faithfulness
    payload_ragas_faith = {
        "model_provider": "openai",
        "model_name": "gpt-4o",
        "metric": "faithfulness",
        "parameters": {
            "user_input": questiony,
            "response": answery,
            "retrieved_contexts": [
                contexty,
            ]
        }
    }
    try:
        response = requests.post(endpoint_url, headers=headers, json=payload_ragas_faith)
        response.raise_for_status()
        result = response.json()
        print("result:", result)
        #### SAVE RAGAS Faithfulness score
        df.loc[ii,'ragas_faith'] = result['score']
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

    ##################################
    ### Factual correctness
    # print("Factual Correctness")
    payload_fact = {
        "model_provider": "openai",
        "model_name": "gpt-4o",
        "metric": "factual_correctness",
        "parameters": {
            "response": str(answery),
            "reference": str(contexty)
        }
    }
    try:
        response = requests.post(endpoint_url, headers=headers, json=payload_fact)
        response.raise_for_status()
        result = response.json()
        print("result:", result)
        #### SAVE Factual correctness score
        df.loc[ii,'ragas_fact'] = result['score']
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

    ##################################
    ### Semantic similarity
    payload_sem_sim = {
        "model_provider": "openai",
        "model_name": "gpt-4o",
        "metric": "semantic_similarity",
        "parameters": {
            "response": answery,
            "reference": contexty
        }
    }
    try:
        response = requests.post(endpoint_url, headers=headers, json=payload_sem_sim)
        response.raise_for_status()
        result = response.json()
        print("result:", result)
        #### SAVE Factual correctness score
        df.loc[ii,'regas_sem_sim'] = result['score']
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

    ##################################
    ### Rouge score
    payload_rogue = {
        "model_provider": "openai",
        "model_name": "gpt-4o",
        "metric": "rouge_score",
        "parameters": {
            "response": answery,
            "reference": contexty
        }
    }
    try:
        response = requests.post(endpoint_url, headers=headers, json=payload_rogue)
        response.raise_for_status()
        result = response.json()
        print("result:", result)
        #### SAVE Factual correctness score
        df.loc[ii,'regas_rogue'] = result['score']
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")


    df.to_csv('C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data\\5_context_question_answer.csv')

0
result: {'metric': 'bleuscore', 'score': 0.16620689217205012}
result: {'metric': 'context_precision_with_reference', 'score': 0.9999999999}
result: {'metric': 'faithfulness', 'score': 1.0}
result: {'metric': 'factual_correctness', 'score': 0.3}
result: {'metric': 'semantic_similarity', 'score': 0.7937536069746418}
result: {'metric': 'rouge_score', 'score': 0.08775137111517366}
1
result: {'metric': 'bleuscore', 'score': 0.21651956746181064}
result: {'metric': 'context_precision_with_reference', 'score': 0.9999999999}
result: {'metric': 'faithfulness', 'score': 1.0}
result: {'metric': 'factual_correctness', 'score': 0.27}
result: {'metric': 'semantic_similarity', 'score': 0.7662673753266493}
result: {'metric': 'rouge_score', 'score': 0.058715596330275226}


In [6]:
df[['similarity_score', 'Cosine', 'sacrebleu_bleu', 'NLTK_bleu', 'chrf',
       'sBURT', 'rouge1', 'rouge2', 'rougeL', 'rougeLsum', 'ragas_bleu',
       'ragas_fact', 'regas_sem_sim', 'regas_rogue']].head()

,similarity_score,Cosine,sacrebleu_bleu,NLTK_bleu,chrf,sBURT,rouge1,rouge2,rougeL,rougeLsum,ragas_bleu,ragas_fact,regas_sem_sim,regas_rogue
0,53.0,0.328335,0.82,3.091681e-07,19.12,0.627521,0.106033,0.055046,0.087751,0.087751,0.166207,0.30,0.793754,0.087751
1,48.0,0.123334,0.86,5.348766e-08,18.28,0.522151,0.084404,0.036832,0.055046,0.055046,0.216520,0.27,0.766267,0.058716
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
